In [2]:
import numpy as np
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from IPython import display
import matplotlib.pylab as plt
import ipywidgets

Convolution

In [3]:
# apply a convolutional kernel to an image matrix
def convolve2d(X, W):
    h, w = X.shape
    m, n = W.shape
    Y = np.zeros((h-m+1, w-n+1))
    for i in range(h-m+1):
        for j in range(w-n+1):
            Y[i][j] = np.sum(X[i:i+m, j:j+n]*W)
    return Y


In [4]:
X = np.array([[1, 2, 3, 4, 5], [6, 7, 8, 9, 10], [6, 7, 8, 9, 10], [16, 17, 18, 19, 20], [21, 22, 23, 24, 25]])
W = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
Y = convolve2d(X, W)
print(Y)

[[8. 8. 8.]
 [8. 8. 8.]
 [8. 8. 8.]]


Output dimension of convolution layer

In [9]:
# make a function that outputs the dimensions of the output of a convolutional layer
def conv_output_shape(h,w, kernel_size=1, stride=1, pad=0, dilation=1):
    from math import floor
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    h_out = floor(((h + (2 * pad) - (dilation * (kernel_size[0] - 1)) - 1) / stride) + 1)
    w_out = floor(((w + (2 * pad) - (dilation * (kernel_size[0] - 1)) - 1) / stride) + 1)
    return h_out, w_out

In [26]:
h = X.shape[0]
w = X.shape[1]
h_out, w_out = conv_output_shape(h,w, kernel_size=1, stride=1, pad=0, dilation=1)
print(h_out, w_out)

h_out, w_out = conv_output_shape(h,w, kernel_size=3, stride=1, pad=1, dilation=1)
print(h_out, w_out)

h_out, w_out = conv_output_shape(6,6, kernel_size=2, stride=2, pad=0, dilation=1)
print(h_out, w_out)

5 5
5 5
3 3


Reseptive field virker ikke!

In [22]:
#Make a function that can determinate the reseptive field of a given pixel (x,y), given a list of kernel sizes, strides, paddings and dilations and number of layers
# and return the receptive field of the pixel (x,y) as min_x, min_y, max_x, max_y 
# such that the reseptive field of the pixel (x,y) is the rectangle with corners (min_x, min_y) and (max_x, max_y)
def receptive_field(x,y,kernel_size, stride, padding, dilation, layers):
    from math import floor
    if type(kernel_size) is not list:
        kernel_size = [kernel_size]*layers
    if type(stride) is not list:
        stride = [stride]*layers
    if type(padding) is not list:
        padding = [padding]*layers
    if type(dilation) is not list:
        dilation = [dilation]*layers
    if len(kernel_size) != layers or len(stride) != layers or len(padding) != layers or len(dilation) != layers:
        print("Error: The number of layers does not match the number of kernel sizes, strides, paddings and dilations")
        return
    min_x = x
    min_y = y
    max_x = x
    max_y = y
    for i in range(layers):
        min_x = floor((min_x - 1) * stride[i] - 2 * padding[i] + dilation[i] * (kernel_size[i] - 1) + 1)
        min_y = floor((min_y - 1) * stride[i] - 2 * padding[i] + dilation[i] * (kernel_size[i] - 1) + 1)
        max_x = floor((max_x - 1) * stride[i] - 2 * padding[i] + dilation[i] * (kernel_size[i] - 1) + 1)
        max_y = floor((max_y - 1) * stride[i] - 2 * padding[i] + dilation[i] * (kernel_size[i] - 1) + 1)
    return min_x, min_y, max_x, max_y

    

In [24]:
rf = receptive_field(1,2,kernel_size=[3,3], stride=[1,2], padding=[0,1], dilation=[1,1], layers=2)
print(rf)
rf = receptive_field(1,2,kernel_size=[3], stride=[1], padding=[1], dilation=[1], layers=1)
print(rf)

(5, 7, 5, 7)
(1, 2, 1, 2)
